# NLP Project: EmoContext

This notebooks explores the use of of `elmo` embedding for our task , EmoContext. 

We have tried the following models in this part:
- LSTM
- CNN
- CNN->LSTM

**Conclusion**:
Using LSTM gave slightly better ouput than the other approaches.


**Note:** You need to upload the training file: `train.txt` in the current folder. 

## Preparing the environment


You need to upload the training file: `train.txt` in the current folder. 
And `devwithoutlabels.txt` needs to be present for testing purposes.

In [0]:
# # Install the latest Tensorflow version.
# !pip install --quiet "tensorflow>=1.7"
# # Install TF-Hub.
# !pip install tensorflow-hub
# !pip install seaborn
# !pip install keras
! pip install tensorboardcolab
! apt-get -qq install -y graphviz && pip3 install -q pydot

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import keras
import os
import pandas as pd
import re
import seaborn as sns
import keras.backend as K
import io
from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.callbacks import *
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, Bidirectional, LSTM
from keras.layers import Embedding, Concatenate, Input, Lambda, Dense, Dropout, Dense
from keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Data

In [0]:
# labels
label2emotion = {0: "others", 1: "happy", 2: "sad", 3: "angry"}
emotion2label = {"others": 0, "happy": 1, "sad": 2, "angry": 3}

def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in  "train" mode] List of labels
    """
    indices = []
    conversations = []
    labels = []
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
#             # Convert multiple instances of . ? ! , to single instance
#             # okay...sure -> okay . sure
#             # okay???sure -> okay ? sure
#             # Add whitespace around such punctuation
#             # okay!sure -> okay ! sure
#             repeatedChars = ['.', '?', '!', ',']
#             for c in repeatedChars:
#                 lineSplit = line.split(c)
#                 while True:
#                     try:
#                         lineSplit.remove('')
#                     except:
#                         break
#                 cSpace = ' ' + c + ' '
#                 line = cSpace.join(lineSplit)

            line = line.strip().split('\t')
            if mode == "train":
              # Train data contains id, 3 turns and label
              label = emotion2label[line[4]]
              labels.append(label)

                
#             conv = ' <eos> '.join(line[1:4])

            # Remove any duplicate spaces
#             duplicateSpacePattern = re.compile(r'\ +')
#             conv = re.sub(duplicateSpacePattern, ' ', conv)

            indices.append(int(line[0]))
            conversations.append(line[1:4])

    if mode == "train":
        return np.array(indices), np.array(conversations), np.array(labels)
    else:
        return np.array(indices), np.array(conversations)

In [0]:
indices, conversations, labels = preprocessData('train.txt', mode='train')

In [0]:
Y = to_categorical(labels)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(conversations,Y, 
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    stratify=Y,
                                                    random_state=32)

In [10]:
x_train.shape

(24128, 3)

In [0]:
# def get_meld_sent():
  
#   import pandas as pd
#   ! wget "https://raw.githubusercontent.com/SenticNet/MELD/master/data/MELD/train_sent_emo.csv"
#   df_train = pd.read_csv('train_sent_emo.csv')
  
#   X = []
#   Y = []
  
#   for i  in range(len(df_train)):
#     big_dialogue = df_train.loc[df_train['Dialogue_ID'] == 2].set_index(['Utterance_ID'])
#     for j in range(len(big_dialogue)-3):
#       x = [big_dialogue.iloc[j]['Utterance'],
#        big_dialogue.iloc[j+1]['Utterance'],
#        big_dialogue.iloc[j+2]['Utterance']]
#       y = big_dialogue.iloc[j+2]['Emotion']

#       X.append(x)
#       Y.append(y)
   
#   #emotion2label = {"neutral": 0, "joy": 1, "sadness": 2, "disgust": 3, "surprise":0}
#   # = {"others": 0, "happy": 1, "sad": 2, "angry": 3}
  
#   #for i in range(len(Y)):
#   #  Y[i] = emotion2label[Y[i]]
  
#   return X, Y
  
# meld_train_x, meld_train_y = get_meld_sent()

In [0]:
# def get_meld_sent():
  
#   import pandas as pd
#   ! wget "https://raw.githubusercontent.com/SenticNet/MELD/master/data/MELD/test_sent_emo.csv"
#   df_train = pd.read_csv('test_sent_emo.csv')
  
#   X = []
#   Y = []
  
#   for i  in range(len(df_train)):
#     big_dialogue = df_train.loc[df_train['Dialogue_ID'] == 2].set_index(['Utterance_ID'])
#     for j in range(len(big_dialogue)-3):
#       x = [big_dialogue.iloc[j]['Utterance'],
#        big_dialogue.iloc[j+1]['Utterance'],
#        big_dialogue.iloc[j+2]['Utterance']]
#       y = big_dialogue.iloc[j+2]['Emotion']

#       X.append(x)
#       Y.append(y)
   
#   #emotion2label = {"neutral": 0, "joy": 1,"anger":3, "sadness": 2, "disgust": 3, "surprise":0}
#   # = {"others": 0, "happy": 1, "sad": 2, "angry": 3}
  
#   #for i in range(len(Y)):
#   #    Y[i] = emotion2label[Y[i]]
  
#   return X, Y
  
# meld_test_x, meld_test_y = get_meld_sent()

In [0]:
# meld_test_x = np.array(meld_test_x)
# meld_train_x = np.array(meld_train_x)
# print(meld_train_x.shape)
# print(meld_test_x.shape)

In [0]:
# from sklearn.preprocessing import LabelBinarizer
# encoder = LabelBinarizer()
# meld_test_y = encoder.fit_transform(meld_test_y)
# meld_test_y.shape

In [0]:
# encoder = LabelBinarizer()
# meld_train_y = encoder.fit_transform(meld_train_y)
# meld_train_y.shape

In [0]:
def get_class_weight(y):
    """

    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights
class_weights = get_class_weight(Y)

In [17]:
class_weights

{0: 0.5044153063955045,
 1: 1.777044543954749,
 2: 1.38019403258283,
 3: 1.3694151834362513}

In [0]:
# Initialize session
sess = tf.Session()
K.set_session(sess)

## Elmo 

In [19]:
# downloading emlo
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


## Models

Choose the model you want to try by specifying the `MODEL_ID`.  

1 -> JUST CNN  
2 -> JUST LSTM  
3 -> BOTH CNN + LSTM   

In [0]:
MODEL_ID = 2


In [0]:
EMBEDDING_DIM = 1024

MAX_SEQUENCE_LENGTH = 10

NUMBER_OF_FILTERS = [128]
KERNEL_SIZE = [4]
lstm_output_size = 128
HIDDEN_LAYER = 128
SIMILARITY_LAYER = 62


In [0]:
def get_embedding(in_str):
  return elmo(tf.squeeze(tf.cast(in_str, tf.string)), signature="default", as_dict=True)["elmo"]

In [0]:
def embedded_cnn(x):
    # Embedding Layer
    emdded = Lambda(get_embedding, output_shape=(None,EMBEDDING_DIM))(x)

    # 1D Conv Layer with multiple possible kernel sizes
    conv_layers = []
    for n_gram, hidden_units in zip(KERNEL_SIZE, NUMBER_OF_FILTERS):
        # 1D Conv with kernel size of n_gram(and with hidden_units of those filters)
        
        
        conv_layer = Conv1D(filters=hidden_units,
                            kernel_size=n_gram,
                            padding='valid',
                            activation='relu')(emdded)

        conv_layer = MaxPooling1D()(conv_layer)
        
        if MODEL_ID ==  3:
          conv_layer = LSTM(lstm_output_size, dropout=0.5)(conv_layer)
          
        if MODEL_ID == 2:
          conv_layer = LSTM(lstm_output_size, dropout=0.5)(emdded)
        
        conv_layers.append(conv_layer)

    if len(conv_layers) == 1:
        return conv_layers[0]
    else:
        # Concatenates conv layers with  different filter sizes
        all_conv_layers_merged = Concatenate()(conv_layers)
        return all_conv_layers_merged

In [24]:
# For first sentence
x_1 = Input(shape=(1,), dtype=tf.string)
f = embedded_cnn(x_1)  # weight x_f here

# For second sentence
x_2 = Input(shape=(1,), dtype=tf.string)
s = embedded_cnn(x_2)  # weight x_s here

# For third sentence
x_3 = Input(shape=(1,), dtype=tf.string)
t = embedded_cnn(x_3)  # weight x_t here

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
fs = Dense(units=SIMILARITY_LAYER,
               activation='relu')(Concatenate()([f, t]))  # M_fs here

# # Similarity between second and third sentence
# # NOTE: I've used a dense layer instead of similarity matrix to get the similarity score
# st = Dense(units=SIMILARITY_LAYER,
#                activation='relu')(Concatenate()([s, t]))  # M_st here

join_layer = Concatenate()([fs, f, s, t])

hidden = Dense(units=HIDDEN_LAYER,
                   activation='relu')(join_layer)
hidden = Dropout(0.5)(hidden)
classifcation = Dense(units=4,
                        activation='softmax')(hidden)

model = Model(inputs=[x_1, x_2, x_3], outputs=classifcation)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 1024)   0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, None, 1024)   0           input_3[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [0]:
# saving the model pics
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_layer_names=True, show_shapes=True)

In [0]:
def for_model(convs):
  X = [[],[],[]]
  
  for con in convs:
    for i in range(3):
      X[i].append(con[i])
      
  return X
x_train_ = for_model(x_train)
x_test_ = for_model(x_test)

In [30]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://e77536d5.ngrok.io


In [0]:
tensorboard = TensorBoardColabCallback(tbc ,write_images=True, write_graph=True, update_freq='batch')

In [0]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=4, \
                          verbose=1, mode='auto', restore_best_weights=True)
checkpointer = ModelCheckpoint(filepath='./weights_all.hdf5', verbose=1, save_best_only=True, save_weights_only=True, monitor='val_loss')

In [0]:
history = model.fit(x_train_,
          y_train,
          validation_data=(x_test_,y_test),
          shuffle=True,
          batch_size=64,
          epochs=30,
          callbacks=[checkpointer,tensorboard,earlystop],
          initial_epoch=0,
          class_weight=class_weights)

In [0]:
model.load_weights('./weights_all.hdf5')

In [0]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [0]:
plot_history(history)

In [0]:
predictions = model.predict(x_test_, batch_size=64)

In [0]:
def getMetrics(predictions, ground):
    """
    FROM: Baseline/starting_kit

    Given predicted labels and the respective ground truth labels, display some metrics
    Input: shape [# of samples, NUM_CLASSES]
        predictions : Model output. Every row has 4 decimal values, with the highest belonging to the predicted class
        ground : Ground truth labels, converted to one-hot encodings. A sample belonging to Happy class will be [0, 1, 0, 0]
    Output:
        accuracy : Average accuracy
        microPrecision : Precision calculated on a micro level. Ref - https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
        microRecall : Recall calculated on a micro level
        microF1 : Harmonic mean of microPrecision and microRecall. Higher value implies better classification
    """
    # [0.1, 0.3 , 0.2, 0.1] -> [0, 1, 0, 0]
    discretePredictions = to_categorical(predictions.argmax(axis=1))

    truePositives = np.sum(discretePredictions * ground, axis=0)
    falsePositives = np.sum(np.clip(discretePredictions - ground, 0, 1), axis=0)
    falseNegatives = np.sum(np.clip(ground - discretePredictions, 0, 1), axis=0)

    print("True Positives per class : ", truePositives)
    print("False Positives per class : ", falsePositives)
    print("False Negatives per class : ", falseNegatives)

    # ------------- Macro level calculation ---------------
    macroPrecision = 0
    macroRecall = 0
    # We ignore the "Others" class during the calculation of Precision, Recall and F1
    for c in range(1, 4):
        precision = truePositives[c] / (truePositives[c] + falsePositives[c])
        macroPrecision += precision
        recall = truePositives[c] / (truePositives[c] + falseNegatives[c])
        macroRecall += recall
        f1 = (2 * recall * precision) / (precision + recall) if (precision + recall) > 0 else 0
        print("Class %s : Precision : %.3f, Recall : %.3f, F1 : %.3f" % (label2emotion[c], precision, recall, f1))

    macroPrecision /= 3
    macroRecall /= 3
    macroF1 = (2 * macroRecall * macroPrecision) / (macroPrecision + macroRecall) if (
                                                                                             macroPrecision + macroRecall) > 0 else 0
    print("Ignoring the Others class, Macro Precision : %.4f, Macro Recall : %.4f, Macro F1 : %.4f" % (
        macroPrecision, macroRecall, macroF1))

    # ------------- Micro level calculation ---------------
    truePositives = truePositives[1:].sum()
    falsePositives = falsePositives[1:].sum()
    falseNegatives = falseNegatives[1:].sum()

    print("Ignoring the Others class, Micro TP : %d, FP : %d, FN : %d" % (
        truePositives, falsePositives, falseNegatives))

    microPrecision = truePositives / (truePositives + falsePositives)
    microRecall = truePositives / (truePositives + falseNegatives)

    microF1 = (2 * microRecall * microPrecision) / (microPrecision + microRecall) if (
                                                                                             microPrecision + microRecall) > 0 else 0
    # -----------------------------------------------------

    predictions = predictions.argmax(axis=1)
    ground = ground.argmax(axis=1)
    accuracy = np.mean(predictions == ground)

    print("Accuracy : %.4f, Micro Precision : %.4f, Micro Recall : %.4f, Micro F1 : %.4f" % (
        accuracy, microPrecision, microRecall, microF1))
    return accuracy, microPrecision, microRecall, microF1
getMetrics(predictions, y_test)

## Testing Upload 

A file called `test.txt` is created which needs to be zipped  and uploaded to the the platform



In [0]:
_, convs = preprocessData('devwithoutlabels.txt',mode='test')
testing = for_model(convs)

In [0]:

predictions = model.predict(testing, batch_size=2, verbose=1)
predictions = predictions.argmax(axis=1)


In [0]:
with io.open('test.txt', "w", encoding="utf8") as fout:
  fout.write('\t'.join(["id", "turn1", "turn2", "turn3", "label"]) + '\n')        
  with io.open('devwithoutlabels.txt', encoding="utf8") as fin:
    fin.readline()
    for lineNum, line in enumerate(fin):
      fout.write('\t'.join(line.strip().split('\t')[:4]) + '\t')
      fout.write(label2emotion[predictions[lineNum]] + '\n')